In [1]:
# lets import everything that we are going to need. 
import pandas as pd
import os
import gc # garbage collection
import numpy as np

In [2]:
print('# File sizes:')
for f in os.listdir('./input'):
     if 'zip' not in f:
        print(f.ljust(30) + str(round(os.path.getsize('./input/' + f) / 1000000, 2)) + 'MB')

# File sizes:
clicks_test.csv               506.0MB
clicks_train.csv              1486.0MB
documents_meta.csv            89.0MB
promoted_content.csv          13.0MB
sample_submission.csv         273.0MB


In [19]:
df_train = pd.read_csv('./input/clicks_train.csv', usecols=['ad_id', 'clicked'])
df_test = pd.read_csv('./input/clicks_test.csv')
print('done reading data input files')

done reading data input files


In [4]:
ad_likelihood = df_train.groupby('ad_id')['clicked'].agg(['count','sum','mean']).reset_index()
mean_clicked = df_train.clicked.mean()
ad_likelihood['likelihood'] = (ad_likelihood['sum'] + 25 * mean_clicked) / (ad_likelihood['count'] + 25)

print(ad_likelihood[:100])

    ad_id  count  sum      mean  likelihood
0       1      2    0  0.000000    0.179301
1       2     22    1  0.045455    0.124279
2       3    161   11  0.068323    0.085167
3       4     32    4  0.125000    0.155108
4       5      1    0  0.000000    0.186197
5       6      1    0  0.000000    0.186197
6       7   3406  985  0.289196    0.288499
7       8     28    4  0.142857    0.166814
8       9      3    0  0.000000    0.172898
9      10     81    9  0.111111    0.130577
10     11     11    0  0.000000    0.134476
11     12     12    2  0.166667    0.184896
12     13      8    1  0.125000    0.177004
13     14     10    1  0.100000    0.166890
14     15      2    0  0.000000    0.179301
15     16      3    1  0.333333    0.208612
16     17     20    2  0.100000    0.152025
17     18      9    0  0.000000    0.142386
18     20     14    0  0.000000    0.124132
19     21     32    0  0.000000    0.084932
20     22      4    0  0.000000    0.166936
21     23     11    2  0.181818 

In [5]:
df_test = df_test.merge(ad_likelihood, how='left')
df_test.fillna(mean_clicked,inplace=True)
df_test.sort_values(['display_id','likelihood'],inplace=True,ascending=False)

print(df_test[:100])

          display_id   ad_id          count           sum      mean  \
32225159    23120126  145293   25321.000000   4516.000000  0.178350   
32225160    23120126  201649   51067.000000   4074.000000  0.079778   
32225161    23120126  292363    7602.000000    589.000000  0.077480   
32225158    23120126   57097   34294.000000   1261.000000  0.036770   
32225155    23120125   89725   19390.000000   3991.000000  0.205828   
32225154    23120125   86281   11425.000000   1852.000000  0.162101   
32225157    23120125  254698     208.000000     21.000000  0.100962   
32225156    23120125  187797  108989.000000   9346.000000  0.085752   
32225150    23120124   83252   18266.000000   5760.000000  0.315340   
32225151    23120124  162994   11430.000000   2585.000000  0.226159   
32225153    23120124  489589     642.000000     99.000000  0.154206   
32225152    23120124  173403  128030.000000  16931.000000  0.132242   
32225147    23120123  208244   27184.000000   7630.000000  0.280680   
322251

In [6]:
output = df_test.groupby(['display_id'])['ad_id'].apply(lambda x:' '.join(map(str,x))).reset_index()

print(output[:100])

output.to_csv('./output/output.csv',index=False)

KeyboardInterrupt: 

In [ ]:
# lets attempt to use a linear classifier 
# and see what the results are...
from sklearn import linear_model

df_doc = pd.read_csv('./input/documents_meta.csv')
df_promoted = pd.read_csv('./input/promoted_content.csv')

df_vectors = df_train.merge(df_promoted, how='left', on='ad_id').merge(df_doc, how='left', on='document_id')
df_vectors.fillna(0.0, inplace=True)

del ad_likelihood
del df_test
del df_doc
del df_promoted
del df_vectors['clicked']

print('len of df_vectors: {}'.format(len(df_vectors)))
print("Sample of df_vectors:\n{}".format(df_vectors[:100]))

X = np.array(df_vectors)
Y = np.array(df_train['clicked'])
print('len of X: {}'.format(len(X)))
print('len of Y: {}'.format(len(Y)))

clf = linear_model.SGDClassifier()
clf.fit(X, Y)
print('finished fitting our classifier.')